In [ ]:
import torch
import numpy as np
import random

from torch.optim import SGD
from torch.optim.lr_scheduler import LinearLR
from torch.utils.data import DataLoader, TensorDataset

from models import MLP
from preprocess import preprocess_UCI_dataset

seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print ("Device: ", device)
print ("Seed: ", seed)

# set random seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

name_to_id_dict = {
    "wine_quality": 186,
    "student_performance": 320,
    "abalone": 1,
    "automobile": 10,
    "auto_mpg": 9,
}


In [ ]:
dataset_name = "wine_quality"
encoding_type = "one_hot"
test_size = 0.2

In [ ]:
X_train, X_test, y_train, y_test, preprocessor = preprocess_UCI_dataset(
    name_to_id_dict[dataset_name], 
    encoding_type, 
    test_size, 
    seed
)
    
# convert to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)

X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

# summary of the dataset
print ("X_train shape: ", X_train.shape)
print ("y_train shape: ", y_train.shape)
print ("X_test shape: ", X_test.shape)
print ("y_test shape: ", y_test.shape)

# create dataloaders
batch_size = 128
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
print ("Batch size: ", batch_size)
print ("Number of iterations per epoch:", len(train_dataloader))

In [ ]:
lr = 1e-3
epochs = 500 # 1e-2. 1e-3. 1e-4
model = MLP(X_train.shape[-1], [32, 32], 1).to(device)
optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)
lr_scheduler = LinearLR(optimizer, end_factor=0.01, start_factor=1, total_iters=epochs*len(train_dataloader))

In [ ]:
# training loop

train_losses = []
test_losses = []

for epoch in range(epochs):

    model.train()
    train_loss = 0

    for i, (X_batch, y_batch) in enumerate(train_dataloader):
        y_pred = model(X_batch)
        loss = torch.nn.functional.mse_loss(y_pred, y_batch, reduction='mean')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        train_loss += loss.item()
    
    train_losses.append(train_loss/len(train_dataloader))

    model.eval()
    y_pred = model(X_test)
    test_loss = torch.mean((y_pred - y_test)**2).item()
    test_losses.append(test_loss)

    if epoch % (epochs//10) == 0:
        print (f"Epoch: {epoch} Train Loss: {train_loss/len(train_dataloader)} Test Loss: {test_loss}")